#**Rapid classification of amorphous-crystalline phase using artificial neural networks and resistivity mapping Ⅰ**

This script is to reproduce the models described in the article entitled of "Electrical Resistivity as a Descriptor for Classification of Amorphous-Crystalline Phases of Alloys," authored by Daegun You, Haitao Zhang, Shraddha Ganorkar, Taeyeop Kim, Jan Schroers, Joost J. Vlassak, and Dongwoo Lee. This is exported version for web-based running, so some minor discrepancies occur with compared to the original script (refer to the original page on github). First, you should download the matrix data file named of 'ML-R-MG.mat' in the original page. Then, it will ask you to upload the matrix again in this page.

In [3]:
import numpy
import numpy as np
from mat4py import loadmat
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from keras.models import Sequential
from keras.layers import Dense
from tensorflow.keras.optimizers import Adam
import pandas as pd
from sklearn import tree
import os
import graphviz
os.environ["PATH"] += os.pathsep + 'C:/Program Files/Graphviz/bin/'




def scaler(X):
    X = (X-np.mean(X, axis=0))/np.std(X, axis=0, ddof=1)
    return X


Load training/test datasets as matrix that includes all features and output (already divided into two sets randomly 20% selected test data) Literature data is also loaded for independent test (transferability test to the multicomponent systems and scales)
###### 0-absolute resistivity, 1-modified resistivity
###### 2-average atomic radii, 3-atomic mismatch
###### 4-heat of mixing, 5-average electronegativity
###### 6-electronegativity difference, 7-number of atom
###### 8-Amorphous/Crystalline label (1/0)

In [2]:
data = loadmat('ML-R-MG.mat')
data_training = data['trainingset_HTEonly']
data_test = data['testset_HTEonly']
data_liter = data['allset_Literature']
data_training = np.asarray(data_training, dtype=np.float64)
data_test = np.asarray(data_test, dtype=np.float64)
data_liter = np.asarray(data_liter, dtype=np.float64)

Scaling input features on the total domain (training+test)
\
(literature data would be scaled within the known scaling domain)

In [3]:
X1 = scaler(np.vstack((data_training[:, :8], data_test[:, :8])))
X2 = scaler(np.vstack((data_training[:, :8], data_test[:, :8], data_liter[:, :8])))

Define training and test input, and their output as well.\
Input example below can be varied on the specific utilization of features as you want

In [4]:
x_train = np.c_[X1[:242,0],X1[:242,3],X1[:242,5]]
x_test = np.c_[X1[242:,0],X1[242:,3],X1[242:,5]]
x_ind = np.c_[X2[302:,0],X2[302:,3],X2[302:,5]]
y_train = data_training[:, 8]
y_test = data_test[:, 8]
y_ind = data_liter[:, 8]

Reshaping dataset as for network flow

In [5]:
x_train = x_train.reshape(242, 3)
x_test = x_test.reshape(60, 3)
x_ind = x_ind.reshape(406, 3)
y_train = y_train.reshape(242,1)
y_test = y_test.reshape(60,1)
y_ind = y_ind.reshape(406, 1)

Early stopping the model training by validation loss

In [6]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=100)
mc = ModelCheckpoint('best_model.h5', monitor='val_loss', mode='min', save_best_only=True)

Set learning rate in Adam optimizer (this value has been already optimized)

In [7]:
opt = Adam(lr=1e-2)

C:\Users\Jaeho\PycharmProjects\ML-Resistivity-MG\venv\lib\site-packages\keras\optimizer_v2\adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


Artificial neural networks of 50 neurons on 1-hidden layer (this has been already optimized)

In [8]:
mdl = Sequential()
mdl.add(Dense(50, input_dim=3, activation='relu'))
mdl.add(Dense(1, activation='sigmoid'))
mdl.compile(optimizer=opt, loss="binary_crossentropy", metrics=['accuracy'])
hst = mdl.fit(x_train, y_train, batch_size=20, epochs=1500,
            validation_data=(x_test, y_test), callbacks=[es, mc])

Epoch 1/1500
13/13 [==============================] - 1s 11ms/step - loss: 0.5722 - accuracy: 0.7107 - val_loss: 0.4514 - val_accuracy: 0.8000
Epoch 2/1500
13/13 [==============================] - 0s 3ms/step - loss: 0.4769 - accuracy: 0.7727 - val_loss: 0.4263 - val_accuracy: 0.8500
Epoch 3/1500
13/13 [==============================] - 0s 3ms/step - loss: 0.4487 - accuracy: 0.7851 - val_loss: 0.4084 - val_accuracy: 0.9167
Epoch 4/1500
13/13 [==============================] - 0s 3ms/step - loss: 0.4311 - accuracy: 0.7893 - val_loss: 0.3820 - val_accuracy: 0.9000
Epoch 5/1500
13/13 [==============================] - 0s 3ms/step - loss: 0.4139 - accuracy: 0.7893 - val_loss: 0.3635 - val_accuracy: 0.9000
Epoch 6/1500
13/13 [==============================] - 0s 3ms/step - loss: 0.4060 - accuracy: 0.7975 - val_loss

Test accuracy

In [9]:
print("\r Test accuracy : %.4f" % (mdl.evaluate(x_test, y_test)[1])*100)

2/2 [==============================] - 0s 3ms/step - loss: 0.4032 - accuracy: 0.9000
 Test accuracy : 0.9000


Test accuracy of literature data for transferability

In [10]:
print("\r Accuracy on literature : %.4f" % (mdl.evaluate(x_ind, y_ind)[1])*100)

13/13 [==============================] - 0s 1000us/step - loss: 1.2947 - accuracy: 0.8054
 Accuracy on literature : 0.8054


In [11]:
node_activation = ["relu", "elu", "selu"]
outcome_activation = ["sigmoid", "tanh", "softmax"]
df_test = pd.DataFrame([[0,0,0],[0,0,0],[0,0,0]])
df_test.columns = node_activation
df_test.index = outcome_activation
df_lit = pd.DataFrame([[0,0,0],[0,0,0],[0,0,0]])
df_lit.columns = node_activation
df_lit.index = outcome_activation




for node in node_activation:
    for outcome in outcome_activation:
        mdl = Sequential()
        mdl.add(Dense(50, input_dim=3, activation='relu'))
        mdl.add(Dense(1, activation='sigmoid'))
        mdl.compile(optimizer=opt, loss="binary_crossentropy", metrics=['accuracy'])
        hst = mdl.fit(x_train, y_train, batch_size=20, epochs=1500,
                    validation_data=(x_test, y_test), callbacks=[es, mc])
        df_test[node][outcome] = (mdl.evaluate(x_test, y_test)[1])*100
        df_lit[node][outcome] = (mdl.evaluate(x_ind, y_ind)[1])*100

Epoch 1/1500
13/13 [==============================] - 0s 9ms/step - loss: 0.5759 - accuracy: 0.6777 - val_loss: 0.4499 - val_accuracy: 0.8167
Epoch 2/1500
13/13 [==============================] - 0s 2ms/step - loss: 0.4614 - accuracy: 0.7479 - val_loss: 0.3610 - val_accuracy: 0.9000
Epoch 3/1500
13/13 [==============================] - 0s 2ms/step - loss: 0.4586 - accuracy: 0.7851 - val_loss: 0.4406 - val_accuracy: 0.8500
Epoch 4/1500
13/13 [==============================] - 0s 2ms/step - loss: 0.3946 - accuracy: 0.8058 - val_loss: 0.3893 - val_accuracy: 0.8667
Epoch 5/1500
13/13 [==============================] - 0s 2ms/step - loss: 0.3867 - accuracy: 0.8058 - val_loss: 0.3727 - val_accuracy: 0.8833
Epoch 6/1500
13/13 [==============================] - 0s 2ms/step - loss: 0.3722 - accuracy: 0.8140 - val_loss

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_42848\1961707586.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test[node][outcome] = (mdl.evaluate(x_test, y_test)[1])*100
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_42848\1961707586.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_lit[node][outcome] = (mdl.evaluate(x_ind, y_ind)[1])*100


KeyboardInterrupt: 

In [ ]:
print(df_test)
print(df_lit)


In [ ]:
print(df_test)
print(df_lit)


In [ ]:
print(df_test)
print(df_lit)

In [ ]:
print(df_test)
print(df_lit)

In [ ]:

clf = tree.DecisionTreeClassifier()
clf = clf.fit(x_train, y_train)
print(clf.score(x_test, y_test)*100)
print(clf.score(x_ind, y_ind)*100)


In [ ]:


dot_data = tree.export_graphviz(clf,   # 의사결정나무 모형 대입
                               out_file = None,  # file로 변환할 것인가
                               feature_names = ["f1","f2","f3"],  # feature 이름
                               class_names = "mg",  # target 이름
                               filled = True,           # 그림에 색상을 넣을것인가
                               rounded = True,          # 반올림을 진행할 것인가
                               special_characters = True)   # 특수문자를 사용하나
graph = graphviz.Source(dot_data)
graph

In [ ]:
graph.render("lala",format="png", view=True)


In [ ]:
X2 = scaler(np.vstack((data_training[:, :8], data_test[:, :8], data_liter[:, :8])))

In [ ]:
print(x_train)